# Kimberlite TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a synthetic kimberlitic model.

In [190]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [191]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [192]:
model_dir = 'kimberlite_data/kimberlite_model.pickle'
grid_dir = '../grids/flightlines_grid_1100pts_16km2_-150m.pickle'
#grid = '../grids/regular_grid_50x50_16km2.pickle'
#data = 'kimberlite_data/regular_grid_50x50_16km2/data.pickle'
data_dir = 'kimberlite_data/flightlines_grid_1100pts_16km/data.pickle'

In [193]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [194]:
with open(grid_dir) as w:
        grid = pickle.load(w)

In [195]:
with open(data_dir) as w:
        data = pickle.load(w)

In [196]:
# observed data and observation points
dobs = data['tfa_obs']
xp = grid['x']
yp = grid['y']
zp = grid['z']
N = grid['N']

### Parameters of the initial model

In [197]:
M = 15 # number of vertices per prism
L = 6 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc'] + 5.
decs = model['dec'] + 5.

# depth to the top, thickness and radius
z0 = model['z0']
#z0 = 150.
dz = 300.
r = 800.

# total magnetization
props = {'magnetization': utils.ang2vec(
        model['intensity']+3., incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = data['main_field']

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

### Limits

In [198]:
# limits for parameters in meters
rmin = 10.
rmax = 1000.
x0min = -1000.
x0max = 1000.
y0min = -1000.
y0max = 1000.
dzmin = 10.
dzmax = 400.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [199]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [200]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [201]:
if grid[9:20] == 'flightlines':
    if incs == model['inc'] and decs == model['dec']:
        mypath = 'tfa_inversion/true_direction_flightlines/r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = 'tfa_inversion/wrong_direction_flightlines/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
else:
    if incs == model['inc'] and decs == model['dec']:
        mypath = 'tfa_inversion/true_direction_regular/r%d_int%d_z0%d_dz%d' % (rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)
    else:
        mypath = 'tfa_inversion/wrong_direction_regular/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [202]:
# output of inversion
kimberlite_inversion = dict()

### Regularization parameters

In [203]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 1000.
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.002  # adjacent radial distances within each prism
a2 = 0.001   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.5     # vertically adjacent origins
a6 = 0.0001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0001     # zero order Tikhonov on thickness of each prism

In [204]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [205]:
itmax = 30
itmax_marq = 10
tol = 1.0e-5     # stop criterion

In [206]:
kimberlite_inversion['x'] = xp
kimberlite_inversion['y'] = yp
kimberlite_inversion['z'] = zp
kimberlite_inversion['observed_data'] = dobs

In [207]:
kimberlite_inversion['inc_dec'] = [incs, decs]
kimberlite_inversion['z0'] = z0
kimberlite_inversion['initial_dz'] = dz
kimberlite_inversion['intial_r'] = r
kimberlite_inversion['initial_estimate'] = model0
kimberlite_inversion['initial_data'] = d0
kimberlite_inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
kimberlite_inversion['regularization'] = alpha
kimberlite_inversion['tol'] = tol
kimberlite_inversion['main_field'] = [data['main_field'][0], data['main_field'][1]]

### Inversion

In [208]:
d_fit, m_est, model_est, phi_list = mfun.levmarq_tf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, inc, dec,
    props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+03   misfit: 2.11126e+05
it:  1   it_marq:  0   lambda: 1e+02   misfit: 1.60172e+05
it:  2   it_marq:  0   lambda: 1e+01   misfit: 5.31831e+04
it:  3   it_marq:  0   lambda: 1e+00   misfit: 1.74827e+04
it:  4   it_marq:  0   lambda: 1e-01   misfit: 5.32648e+03
it:  5   it_marq:  0   lambda: 1e-02   misfit: 2.48830e+03
it:  6   it_marq:  0   lambda: 1e-03   misfit: 1.58280e+03
it:  7   it_marq:  0   lambda: 1e-04   misfit: 5.19017e+04
it:  7   it_marq:  1   lambda: 1e-03   misfit: 1.18331e+03
it:  8   it_marq:  0   lambda: 1e-04   misfit: 5.23189e+04
it:  8   it_marq:  1   lambda: 1e-03   misfit: 5.23194e+04
it:  8   it_marq:  2   lambda: 1e-02   misfit: 7.87993e+02
it:  9   it_marq:  0   lambda: 1e-03   misfit: 5.26201e+04
it:  9   it_marq:  1   lambda: 1e-02   misfit: 5.24152e+04
it:  9   it_marq:  2   lambda: 1e-01   misfit: 5.24145e+04
it:  9   it_marq:  3   lambda: 1e+00   misfit: 7.53947e+02
it: 10   it_marq:  0   lambda: 1e-01   misfit: 6.24157e+

# Results

In [209]:
kimberlite_inversion['data_fit'] = d_fit
kimberlite_inversion['estimate'] = m_est
kimberlite_inversion['prisms'] = model_est
kimberlite_inversion['objective'] = phi_list
kimberlite_inversion['residual'] = dobs - d_fit

In [210]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(kimberlite_inversion, f)